In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
from common_import import *


def check_config_experiment(whitelist_checkpoint):
    user = getpass.getuser()
    motionbert_root = f'/home/{user}/codes/MotionBERT'
    for checkpoint in whitelist_checkpoint:
        experiment_path = f'{motionbert_root}/experiments/{checkpoint}.txt'
        config_path = f'{motionbert_root}/configs/pose3d/{checkpoint}.yaml'
        print(f'{checkpoint}')
        if os.path.exists(experiment_path): print(f'Experiment path: {experiment_path}')
        else:
            print(f'[No experiment path] {experiment_path}')
            with open(experiment_path, 'w') as f: pass
        if os.path.exists(config_path): print(f'Config# path: {config_path}')
        else: print(f'[No config path] {config_path}')

def update_result_dict_pkl(blacklist_checkpoint=[]):
    if os.path.exists('result_dict.pkl'):
        print('Loading result_dict.pkl')
        result_dict = readpkl('result_dict.pkl')
    else:
        result_dict = {}
    result_root = '/home/hrai/codes/MotionBERT/saved_results'
    checkpoint_list = os.listdir(result_root)
    for checkpoint in tqdm(checkpoint_list):
        if checkpoint in blacklist_checkpoint: continue # skip
        if checkpoint not in result_dict.keys(): result_dict[checkpoint] = {} # create new key if not exists
        checkpoint_root = os.path.join(result_root, checkpoint)
        subset_list = glob(checkpoint_root+'/*.pkl')
        for item in subset_list:
            subset = os.path.basename(item).split('.pkl')[0]
            if subset in result_dict[checkpoint].keys():
                #print(f'{subset} exists')
                continue
            result = readpkl(item)
            result_dict[checkpoint][subset] = {'e1': result['e1'], 'e2': result['e2']}
        # remove key if not in subset_list
        for key in list(result_dict[checkpoint].keys()):
            if key not in [os.path.basename(item).split('.pkl')[0] for item in subset_list]:
                del result_dict[checkpoint][key]

    savepkl(result_dict, 'result_dict.pkl')

### Evaluate single experiment

In [17]:
checkpoint_name = 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03'
check_config_experiment([checkpoint_name])

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03
Experiment path: /home/hrai/codes/MotionBERT/experiments/MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03.txt
Config# path: /home/hrai/codes/MotionBERT/configs/pose3d/MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03.yaml


In [31]:
# load model
args, opts = get_opt_args_from_model_name(checkpoint_name, verbose=False)
model_pos, chk_filename, checkpoint = load_model(opts, args)
# load dataset
subset = '3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_ALL_TRAIN'
subset_path = f'/home/{user}/codes/MotionBERT/data/motion3d/yaml_files/{subset}.yaml'
print(subset_path)
if 'PCL' in subset: args.fix_orientation_pred = True # load dataset
args.subset_list = [subset]
train_loader, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)
e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False)

# save results
save = True
if save:
    save_folder = f'/home/hrai/codes/MotionBERT/saved_results/{checkpoint_name}'
    if not os.path.exists(save_folder): os.makedirs(save_folder)
    save_path = os.path.join(save_folder, f'{subset}.pkl')
    results_dict = {'results_all': results_all, 'total_result_dict': total_result_dict, 'e1': e1, 'e2': e2} # 'gts_all': gts_all, 'inputs_all': inputs_all, 
    savepkl(results_dict, save_path)

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03


Loading checkpoint checkpoint/pose3d/MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03/best_epoch.bin
/home/hrai/codes/MotionBERT/data/motion3d/yaml_files/3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_ALL_TRAIN.yaml
Loading dataset...
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_3D_CANONICAL_REVOLUTE-TEST_ALL_TRAIN
[overwrite: False] ==> Loading 3DHP source_list...
[overwrite: False] ==> Loading 3DHP cam_param...
[overwrite: False] ==> Loading 3DHP cam_3d...
[overwrite: False] ==> Loading 3DHP cam_3d_canonical_revolute...
[overwrite: False] ==> Loading 3DHP img_2d_canonical_revolute...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 245/245 [01:31<00:00,  2.67it/s]


No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 178.31979446553666 mm
Protocol #2 Error (P-MPJPE): 128.27692643796124 mm
----------------------------------------


In [33]:
update_result_dict_pkl()

Loading result_dict.pkl


  0%|          | 0/88 [00:00<?, ?it/s]

100%|██████████| 88/88 [00:00<00:00, 199.06it/s]


In [32]:
# update result_dict
remove_subset_from_result_dict = True
if remove_subset_from_result_dict:
    result_dict = readpkl('result_dict.pkl')
    try: del result_dict[checkpoint_name][subset]
    except: print('subset not found')
    savepkl(result_dict, 'result_dict.pkl')

subset not found


### Evaluate and save all experiments

In [ ]:
result_list = os.listdir('saved_results')
checkpoint_list = os.listdir('checkpoint/pose3d')
whitelist_checkpoint = []
for item in os.listdir('configs/pose3d'):
    checkpoint = item.split('.yaml')[0]
    if 'cam_no_factor' not in checkpoint: continue
    if checkpoint in blacklist_checkpoint: continue
    if checkpoint not in checkpoint_list:
        print(f'[Need to be trained] {checkpoint}')
        continue
    if checkpoint not in experiment_list:
        whitelist_checkpoint.append(checkpoint)
        # create empty file
        with open(f'experiments/{checkpoint}.txt', 'w') as f:
            pass
    else:
        subset_list = []
        with open(f'experiments/{checkpoint}.txt', 'r') as f:
            lines = f.readlines()
            for line in lines:
                subset = line.split('\n')[0]
                subset_list.append(subset)
        if len(subset_list) == 0:
            print(f'[No subset fou  d] {checkpoint}')
            whitelist_checkpoint.append(checkpoint)
        for subset in subset_list:
            if not os.path.exists(f'saved_results/{checkpoint}/{subset}.pkl'):
                print(f'[Result not found] {checkpoint} - {subset}')
                if checkpoint not in whitelist_checkpoint:
                    whitelist_checkpoint.append(checkpoint)

print('\nCheckpoints to evaluate:')
whitelist_checkpoint

In [6]:
whitelist_checkpoint = [
    # 'MB_train_h36m_gt_cam_no_factor',
    # 'MB_train_h36m_gt_cam_no_factor_tr_s1_ts_s5678',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s1_ts_s5678',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_revolute_no_Rz_input_centering_tr_s1_ts_s5678',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_input_centering_tr_s1_ts_s5678',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_input_centering',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original',
    # 'MB_train_h36m_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s1_ts_s5678',
    # 'MB_train_fit3d_gt_cam_no_factor_tr_s03',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_tr_s03',
    # 'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_tr_s03',
    #'MB_train_fit3d_gt_cam_no_factor_ts_s4710',
    'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710',
    #'MB_train_fit3d_gt_cam_no_factor_input_from_canonical_pcl_original_ts_s4710',
]

# check experiment file
check_config_experiment(whitelist_checkpoint)

MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710
Experiment path: /home/hrai/codes/MotionBERT/experiments/MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710.txt
Config# path: /home/hrai/codes/MotionBERT/configs/pose3d/MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710.yaml


In [7]:
experiment_root = f'/home/{user}/codes/MotionBERT/experiments'
for item in glob(experiment_root+'/*.txt'):
    checkpoint_name = os.path.basename(item).split('.txt')[0]
    # check if in whitelist
    if checkpoint_name in blacklist_checkpoint: continue
    if len(whitelist_checkpoint) > 0 and checkpoint_name not in whitelist_checkpoint: continue
    # filtering
    if 'cam_no_factor' not in checkpoint_name: continue
    # get save folder
    save_folder = f'/home/hrai/codes/MotionBERT/saved_results/{checkpoint_name}'
    if not os.path.exists(save_folder): os.makedirs(save_folder)

    # load model
    args, opts = get_opt_args_from_model_name(checkpoint_name, verbose=False)
    model_pos, chk_filename, checkpoint = load_model(opts, args)

    # load subset list
    subset_list = []
    with open(item, 'r') as f:
        lines = f.readlines()
        for line in lines:
            subset = line.strip()
            subset_list.append(subset)

    for subset in subset_list:
        if 'PCL' in subset: args.fix_orientation_pred = True
        # get save path
        save_path = os.path.join(save_folder, f'{subset}.pkl')
        # check if exists
        if os.path.exists(save_path):
            print(f'{save_path} exists')
            continue
        # evaluation
        try:
            # load dataset
            args.subset_list = [subset]
            train_loader, test_loader, _, _, datareader = load_dataset(args, use_new_datareader=True)
            # evaluation
            e1, e2, results_all, inputs_all, gts_all, total_result_dict = evaluate(args, model_pos, test_loader, datareader, checkpoint, only_one_batch=False)
            # save results
            results_dict = {'results_all': results_all, 'total_result_dict': total_result_dict, 'e1': e1, 'e2': e2} # 'gts_all': gts_all, 'inputs_all': inputs_all, 
            savepkl(results_dict, save_path)
        except Exception as e:
            print(f'Error: {e}')


MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710
Loading checkpoint checkpoint/pose3d/MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710/best_epoch.bin
/home/hrai/codes/MotionBERT/saved_results/MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710/H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_REVOLUTE.pkl exists
/home/hrai/codes/MotionBERT/saved_results/MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710/H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_REVOLUTE-TR_S1_TS_S5678.pkl exists
/home/hrai/codes/MotionBERT/saved_results/MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710/3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_REVOLUTE-TEST_TS1_6.pkl exists
/home/hrai/codes/MotionBERT/saved_results/MB_train_fit3d_gt_cam_no_factor_input_from_canonical_3d_revolute_input_centering_ts_s4710/3DHP-GT-CAM_NO_FACTOR-INP

100%|██████████| 133/133 [00:46<00:00,  2.89it/s]


No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 118.48518790699215 mm
Protocol #2 Error (P-MPJPE): 91.32111901138943 mm
----------------------------------------
Loading dataset...
H36M-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TR_S1_TS_S5678
[overwrite: False] ==> Loading H36M source_list...
[overwrite: False] ==> Loading H36M cam_param...
[overwrite: False] ==> Loading H36M cam_3d...
[overwrite: False] ==> Loading H36M cam_3d_canonical_pcl_with_Rz...
[overwrite: False] ==> Loading H36M img_2d_canonical_pcl_with_Rz...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 323/323 [01:50<00:00,  2.92it/s]


No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 112.60223946714903 mm
Protocol #2 Error (P-MPJPE): 83.44745737824775 mm
----------------------------------------
Loading dataset...
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_TS1_6
[overwrite: False] ==> Loading 3DHP source_list...
[overwrite: False] ==> Loading 3DHP cam_param...
[overwrite: False] ==> Loading 3DHP cam_3d...
[overwrite: False] ==> Loading 3DHP cam_3d_canonical_pcl_with_Rz...
[overwrite: False] ==> Loading 3DHP img_2d_canonical_pcl_with_Rz...
INFO: Testing
No epoch information in the checkpoint


...

No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 156.41882371921932 mm
Protocol #2 Error (P-MPJPE): 111.60569914796697 mm
----------------------------------------
Loading dataset...
3DHP-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TEST_ALL_TRAIN
[overwrite: False] ==> Loading 3DHP source_list...
[overwrite: False] ==> Loading 3DHP cam_param...
[overwrite: False] ==> Loading 3DHP cam_3d...
[overwrite: False] ==> Loading 3DHP cam_3d_canonical_pcl_with_Rz...
[overwrite: False] ==> Loading 3DHP img_2d_canonical_pcl_with_Rz...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 245/245 [01:24<00:00,  2.91it/s]


No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 165.80987836368877 mm
Protocol #2 Error (P-MPJPE): 114.83711528336457 mm
----------------------------------------
Loading dataset...
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TS_S4710
data/motion3d/yaml_files/FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TS_S4710.yaml does not exist... generating yaml file
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TS_S4710
FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TS_S4710
/home/hrai/codes/MotionBERT/data/motion3d/yaml_files/FIT3D-GT-CAM_NO_FACTOR-INPUT_FROM_CANONICAL_PCL_WITH_RZ-TS_S4710.yaml
[overwrite: False] ==> Loading FIT3D source_list...
[overwrite: False] ==> Loading FIT3D cam_param...
[overwrite: False] ==> Loading FIT3D cam_3d...
[overwrite: False] ==> Loading FIT3D cam_3d_canonical_pcl_with_Rz...
[overwrite: False] ==> Loading FIT3D img_2d_canonical_pcl_with_Rz...
INFO: Testing
No epoch information in the checkpoint


100%|██████████| 157/157 [00:54<00:00,  2.90it/s]


No eval_keypoint. Use part list
Part: whole
Protocol #1 Error (MPJPE): 31.348291051425402 mm
Protocol #2 Error (P-MPJPE): 20.281340279687633 mm
----------------------------------------


### Reset the result_dict for whitelist_checkpoint

In [8]:
result_dict = readpkl('result_dict.pkl')
for item in whitelist_checkpoint:
    try: del result_dict[item]
    except: print(f'[{item}] not found')
savepkl(result_dict, 'result_dict.pkl')

### Cherrypick e1, e2 -> result_dict.pkl

In [9]:
blacklist_checkpoint = ['MB_train_h36m_gt_cam_no_factor_input_from_canonical_3d_same_z_s15678_tr_54138969_ts_others']
update_result_dict_pkl(blacklist_checkpoint)

Loading result_dict.pkl


  0%|          | 0/90 [00:00<?, ?it/s]

100%|██████████| 90/90 [00:02<00:00, 32.46it/s]


### Delete gts_all, inputs_all to reduce the file size

In [ ]:
experiment_root = f'/home/{user}/codes/MotionBERT/experiments'
for item in tqdm(glob(experiment_root+'/*.txt')):
    checkpoint_name = os.path.basename(item).split('.txt')[0]
    save_folder = f'/home/hrai/codes/MotionBERT/saved_results/{checkpoint_name}'
    subset_list = []
    with open(item, 'r') as f:
        lines = f.readlines()
        for line in lines:
            subset = line.strip()
            subset_list.append(subset)

    for subset in subset_list:
        save_path = os.path.join(save_folder, f'{subset}.pkl')
        if not os.path.exists(save_path): continue
        results_dict = readpkl(save_path)
        #print(checkpoint_name, subset, results_dict.keys())
        if 'gts_all' in results_dict.keys(): del results_dict['gts_all']
        if 'inputs_all' in results_dict.keys(): del results_dict['inputs_all']
        savepkl(results_dict, save_path)

  0%|          | 0/97 [00:00<?, ?it/s]

 31%|███       | 30/97 [10:41<22:45, 20.39s/it]